In [1]:
import pickle
import numpy as np
from typing import Dict
import os
import xarray as xr
%load_ext autoreload
%autoreload 2
from basic_functions import *
import itertools
import plotly.graph_objects as go


In [8]:
def plot_3D_echogram(src_path:str="../data/acoustic_data/18kHz/processed_data/18kHz_v1_summer.pkl") :
    ni = 0
    with open(src_path, 'rb') as pkl_file:
        # read data in pkl as stream
        while True:
            try:
                if ni>0 :
                    break
                ni+=1
                print("hey")
                data_title, data_dict = pickle.load(pkl_file)
                print(data_dict.keys())
                depth=data_dict["DEPTH"][::5].astype(np.float32)
                longitude = data_dict["LONGITUDE"][::60].astype(np.float32)
                latitude = data_dict["LATITUDE"][::60].astype(np.float32)
                chan = data_dict["CHANNEL"]
                n = latitude.shape[0]
                d=depth.shape[0]
            
                if len(chan)>1 :
                    sv = data_dict["Sv"][::60, ::5, 0]
                else : 
                    sv = data_dict["Sv"][::60, ::5]

                 # Construction des points 3D
                lon_flat = np.repeat(longitude, d)  # (n*d,)
                lat_flat = np.repeat(latitude, d)  # (n*d,)
                depth_flat = np.tile(depth, n)  # (n*d,)
                Sv_flat = sv.ravel()  # (n*d,)

                print(f"Shapes aplatis -> lon_flat: {lon_flat.shape}, lat_flat: {lat_flat.shape}, depth_flat: {depth_flat.shape}, Sv_flat: {Sv_flat.shape}")

                # Création du scatter 3D
                fig = go.Figure(
                    data=[go.Scatter3d(
                        x=lon_flat,
                        y=lat_flat,
                        z=depth_flat,
                        mode='markers',
                        marker=dict(
                            size=2,
                            color=Sv_flat,
                            colorscale='Viridis',
                            colorbar=dict(title="Sv (dB)"),
                            opacity=0.8
                        )
                    )]
                )

                # Mise en page
                fig.update_layout(
                    title="Échogramme 3D (Sonar)",
                    scene=dict(
                        xaxis_title="Longitude",
                        yaxis_title="Latitude",
                        zaxis_title="Profondeur (m)",
                        zaxis=dict(autorange="reversed")  # Inverser l'axe Z (profondeur)
                    )
                )

                fig.show()
                
            except EOFError:
                print("End of file.")
                break

In [9]:
plot_3D_echogram()

hey
dict_keys(['DEPTH', 'TIME', 'SEASON', 'LONGITUDE', 'LATITUDE', 'CHANNEL', 'Sv', 'DAY'])
Shapes aplatis -> lon_flat: (12288,), lat_flat: (12288,), depth_flat: (12288,), Sv_flat: (12288,)
